<a href="https://colab.research.google.com/github/namakmurtaza/Predicting-disease-outbreak/blob/main/Untitled8_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import plotnine as pltn
import seaborn as sns

In [ ]:
hep = pd.read_csv("/content/hepatitis.csv")
measles = pd.read_csv("/content/measles.csv")
mump = pd.read_csv("/content/mumps.csv")
polio = pd.read_csv("/content/polio.csv")
pert = pd.read_csv("/content/pertussis.csv")
rub = pd.read_csv("/content/rubella.csv")
smlpx = pd.read_csv("/content/smallpox.csv")

In [ ]:
disease = hep.append(measles)
disease = disease.append(mump)
disease = disease.append(pert)
disease = disease.append(polio)
disease = disease.append(rub)
disease = disease.append(smlpx)


In [ ]:
# Combine dataframes measles, hepatitis, mumps, pertussis, polio, rubella, smallpox into one dataframe 'all'
disease = pd.concat([measles, hep, mump, pert, polio, rub, smlpx])

# Rename 'week' to 'date'
disease = disease.rename(columns={'week': 'date'})

# Convert 'date' to string
disease['date'] = disease['date'].astype(str)

# Extract the year from 'date'
disease['year'] = disease['date'].str.extract(r'([0-9][0-9][0-9][0-9])')

# Extract the week from 'date'
disease['week'] = disease['date'].str.extract(r'(\d[0-9])')

# Convert 'date', 'week', and 'year' to integers
disease['date'] = disease['date'].astype(int)
disease['week'] = disease['week'].astype(int)
disease['year'] = disease['year'].astype(int)

# Display the first 10 rows of the combined data
print(disease.head(10))

In [ ]:
# Create the 'year' column
disease['year'] = (disease['week'] // 100).astype(int)

# Create the 'weekNb' column
disease['weekNb'] = disease['week'] - disease['year'] * 100

# Create the 'tempsContinue' column
disease['tempsContinue'] = disease['year'].astype(float) + disease['weekNb'].astype(float) / 52

# Aggregate data
toPlot = disease.groupby(['tempsContinue', 'disease'])['incidence_per_capita'].sum().reset_index()

# Sort by 'tempsContinue'
toPlot = toPlot.sort_values(by='tempsContinue')

# Plot data using seaborn
for disease in toPlot['disease'].unique():
    data_subset = toPlot[toPlot['disease'] == disease]

    plt.figure(figsize=(10, 5))
    sns.barplot(x='tempsContinue', y='incidence_per_capita', data=data_subset, hue='incidence_per_capita')

    plt.title(disease)
    plt.ylim(0, data_subset['incidence_per_capita'].max())
    plt.legend([])
    plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Load your data into a Pandas DataFrame (replace 'your_data.csv' with your actual data file)
data1 = pd.read_csv('/content/hepatitis.csv')

# Perform groupby operation
toPlot = data1.groupby(['state_name', 'disease'])['incidence_per_capita'].sum().reset_index()
toPlot = data1.groupby(['state_name', 'disease'])['incidence_per_capita'].sum().reset_index()

# Create a boxplot using Seaborn
plt.figure(figsize=(10, 5))
sns.boxplot(x='disease', y='incidence_per_capita', data=toPlot, hue='disease', palette='Set3')
plt.ylim(0, 0.1)
plt.title('Incidence per Capita by Disease')
plt.xlabel('Disease')
plt.ylabel('Incidence per Capita')
plt.legend(title='Disease', loc='upper right')
plt.show()


In [ ]:
import numpy as np

import seaborn as sns
import re

import pandas as pd

# Replace non-finite values (NaN) with a default value, for example, 0
disease['cases'] = disease['cases'].fillna(0)

# Convert 'cases' column to integers
disease['cases'] = disease['cases'].astype(int)

# Calculate 'year_mean' by grouping and averaging 'cases'
disease_decade_mean = disease.groupby(['year', 'disease', 'week'])['cases'].mean().reset_index()

# Remove missing values
disease_decade_mean = disease_decade_mean.dropna()

# Filter out rows with year_mean equal to 0
disease_decade_mean = disease_decade_mean[disease_decade_mean['year_mean'] != 0]

# Initialize the 'decade' column to 0
disease_decade_mean['decade'] = 0

# Convert 'year' to strings
disease_decade_mean['year'] = disease_decade_mean['year'].astype(str)

# Create the 'decade' column by extracting the decade part
disease_decade_mean['decade'] = disease_decade_mean['year'].apply(lambda x: str(int(re.search(r'[0-9][0-9][0-9]', x).group(0)) * 10))

# Convert 'year' to integers
disease_decade_mean['year'] = disease_decade_mean['year'].astype(int)

# Group by decade, disease, year, and week, and calculate the mean of year_mean
disease_decade_mean = disease_decade_mean.groupby(['decade', 'disease', 'year', 'week'])['year_mean'].mean().reset_index()

# Convert 'decade' to a categorical variable
disease_decade_mean['decade'] = disease_decade_mean['decade'].astype('category')

# Calculate the logarithm of 'mean' column
disease_decade_mean['logmean'] = np.log10(disease_decade_mean['year_mean'])

# Rename 'disease' to 'Disease'
disease_decade_mean = disease_decade_mean.rename(columns={'disease': 'Disease'})

# Create a facet grid plot using seaborn
sns.set(style="whitegrid")
g = sns.FacetGrid(disease_decade_mean, col='decade', hue='Disease', col_wrap=3)
g.map(sns.kdeplot, 'logmean', fill=True, common_norm=False)
g.map(sns.rugplot, 'logmean')
g.add_legend()

# Set plot labels and titles
g.set_xlabels('Logarithmic Transformation of Mean Disease Cases')
g.set_ylabels('Density')
g.set_titles("Disease Outbreak Through The Decades")
g.fig.suptitle("Disease Outbreak Through The Decades", fontsize=18, fontweight='bold', y=1.02)

plt.show()

# Section 2: Number of Disease Cases (Annually)
disease_year_mean = disease_decade_mean.groupby(['year', 'Disease'])['year_mean'].mean().reset_index()



MEASLES

In [ ]:
measles.head()

In [ ]:
measles.info()

In [ ]:
sns.boxplot(measles['cases'])

In [ ]:
sns.boxplot(measles['incidence_per_capita'])

In [ ]:
plt.scatter(measles['cases'], measles['incidence_per_capita'])

In [ ]:
plt.scatter(measles['cases'], measles['incidence_per_capita'])

In [ ]:
plt.matshow(measles.corr())
plt.colorbar()
plt.show()

In [ ]:
mump.head()

MUMP disease

In [ ]:
mump.info()

In [ ]:
sns.boxplot(mump['cases'])

In [ ]:
sns.boxplot(mump['incidence_per_capita'])

In [ ]:
plt.scatter(mump['cases'], mump['incidence_per_capita'])

In [ ]:
plt.scatter(mump['cases'], mump['incidence_per_capita'])

In [ ]:
plt.matshow(mump.corr())
plt.colorbar()
plt.show()

Hepatitis

In [ ]:
hep.head()

In [ ]:
hep.info()

In [ ]:
sns.boxplot(hep['cases'])

In [ ]:
sns.boxplot(hep['incidence_per_capita'])

In [ ]:
plt.scatter(hep['cases'], hep['incidence_per_capita'])

In [ ]:
plt.scatter(hep['cases'], hep['incidence_per_capita'])

In [ ]:
plt.matshow(hep.corr())
plt.colorbar()
plt.show()

Pertussis

In [ ]:
pert.head()

In [ ]:
pert.info()

In [ ]:
sns.boxplot(pert['cases'])

In [ ]:
sns.boxplot(pert['incidence_per_capita'])

In [ ]:
plt.scatter(pert['cases'], pert['incidence_per_capita'])

In [ ]:
plt.matshow(pert.corr())
plt.colorbar()
plt.show()

Polio

In [ ]:
polio.head()

In [ ]:
polio.info()

In [ ]:
plt.matshow(polio.corr())
plt.colorbar()
plt.show()

Rubella

In [ ]:
rub.head()

In [ ]:
rub.info()

In [ ]:
sns.boxplot(rub['cases'])

In [ ]:
sns.boxplot(rub['incidence_per_capita'])

In [ ]:
plt.scatter(rub['cases'], rub['incidence_per_capita'])

In [ ]:
plt.matshow(rub.corr())
plt.colorbar()
plt.show()

Smallpox

In [ ]:
smlpx.head()

In [ ]:
smlpx.info()

In [ ]:
sns.boxplot(smlpx['cases'])

In [ ]:
sns.boxplot(smlpx['incidence_per_capita'])

In [ ]:
plt.scatter(smlpx['cases'], smlpx['incidence_per_capita'])

In [ ]:
plt.matshow(smlpx.corr())
plt.colorbar()
plt.show()

.................................................................................................................................................................................................................................................

.................................................................................................................................................................................................................................................

# To group regions with similar characterstics using K-Means Clustering

# MEASLES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
data= pd.read_csv('/content/measles.csv')

In [ ]:
measles.head()

In [ ]:
features = data[['incidence_per_capita', 'cases']]

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)


In [ ]:
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(scaled_features)

In [ ]:
data['Cluster'] = kmeans.labels_

In [ ]:
plt.scatter(data['incidence_per_capita'], data['cases'], c=data['Cluster'], cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')
plt.title('Clusters of Regions with Similar Infectious Disease Characteristics')
plt.xlabel('incidence_per_capita')
plt.ylabel('cases')
plt.show()

-->Silhouette Score

The Silhouette Score measures how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

data= pd.read_csv('/content/measles.csv')
data = np.random.rand(100, 2)

kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(data)

silhouette_avg = silhouette_score(data, labels)



In [ ]:
print(f"Silhouette Score: {silhouette_avg}")

-->Davies-Bouldin Index

The Davies-Bouldin Index (DBI) is a metric for evaluating the quality of clusters in a dataset. It measures the compactness and separation between clusters. The lower the Davies-Bouldin Index, the better the clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import numpy as np

data= pd.read_csv('/content/measles.csv')
data = np.random.rand(100, 2)

# Fit K-means model
kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(data)

# Calculate Davies-Bouldin Index
dbi_score = davies_bouldin_score(data, labels)


In [ ]:
print(f"Davies-Bouldin Index: {dbi_score}")

# SMALLPOX

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
data= pd.read_csv('/content/smallpox.csv')

In [ ]:
data.head()

In [ ]:
features = data[['incidence_per_capita', 'cases']]

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(scaled_features)

In [ ]:
data['Cluster'] = kmeans.labels_

In [ ]:
plt.scatter(data['incidence_per_capita'], data['cases'], c=data['Cluster'], cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')
plt.title('Clusters of Regions with Similar Infectious Disease Characteristics')
plt.xlabel('incidence_per_capita')
plt.ylabel('cases')
plt.show()

Silhouette Score

The Silhouette Score measures how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

data= pd.read_csv('/content/smallpox.csv')
data = np.random.rand(100, 2)

kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(data)

silhouette_avg = silhouette_score(data, labels)

In [ ]:
print(f"Silhouette Score: {silhouette_avg}")

-->Davies-Bouldin Index



The Davies-Bouldin Index (DBI) is a metric for evaluating the quality of clusters in a dataset. It measures the compactness and separation between clusters. The lower the Davies-Bouldin Index, the better the clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import numpy as np

data= pd.read_csv('/content/smallpox.csv')
data = np.random.rand(100, 2)

# Fit K-means model
kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(data)

# Calculate Davies-Bouldin Index
dbi_score = davies_bouldin_score(data, labels)


In [ ]:
print(f"Davies-Bouldin Index: {dbi_score}")

# RESULT

# Silhouette Score

MEASLES Silhouette Score: 0.38670076366306


SMALLPOX Silhouette Score: 0.3986899415621677

Silhouette Score of "SMALLPOX" is highest than that of Measles

# Davies-Bouldin Index

MEASLES Davies-Bouldin Index: 0.8356174660890149

SMALLPOX Davies-Bouldin Index: 0.8016217748774546

Davies-Bouldin Index of "SMALLPOX" is highest than that of Measles

.................................................................................................................................................................................................................................................

# To identify hotspots of disease outbreak using spatial analysis

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
gdf = gpd.read_file('/content/measles_spatial.csv')

In [ ]:
gdf.head()

In [ ]:
g = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.longitude, gdf.latitude))
g.crs={'init': 'epsg:4326'}
g.head()

In [ ]:
gdf.plot(markersize=5, color='blue', alpha=0.5)
plt.title('Original Spatial Dataset')
plt.xlabel("latitude")
plt.ylabel("longitude")
plt.show()

In [ ]:
coordinates = gdf['geometry'].apply(lambda geom: [geom.x, geom.y]).tolist()


In [ ]:
coords_array = np.array(coordinates)

In [ ]:
scaler = StandardScaler()
scaled_coords = scaler.fit_transform(coords_array)

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust parameters based on your dataset
clusters = dbscan.fit_predict(scaled_coords)


In [ ]:
gdf['Cluster'] = clusters

In [ ]:
gdf.plot(column='Cluster', cmap='viridis', markersize=30, alpha=0.5, legend=True)
plt.title('Spatial Clusters (Hotspots) of Disease Outbreak')
plt.xlabel("latitude")
plt.ylabel("longitude")
plt.show()
